In [2]:
!pip install python-docx PyPDF2 ebooklib


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.5 MB/s eta 0:00:00


In [3]:
from pathlib import Path
import docx
import PyPDF2
from ebooklib import epub


In [14]:
!apt-get update -y
!apt-get install -y --no-install-recommends espeak-ng espeak ffmpeg libsndfile1

# Core Python packages
!pip install --quiet torch torchvision torchaudio
!pip install --quiet gradio transformers accelerate datasets phonemizer

# Install Coqui TTS from GitHub (Python 3.12 compatible)
!pip install --quiet git+https://github.com/coqui-ai/TTS


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 3,917 B in 2s (1,942 B/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (

In [15]:
import os
import re
import torch
import soundfile as sf
import numpy as np
import gradio as gr
from transformers import AutoTokenizer, AutoModelForCausalLM
from TTS.api import TTS


In [16]:
GRANITE_MODEL = "ibm-granite/granite-3b-code-instruct"  # Example model, replace with <2B
granite_tokenizer = AutoTokenizer.from_pretrained(GRANITE_MODEL)
granite_model = AutoModelForCausalLM.from_pretrained(
    GRANITE_MODEL,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto"
)

def refine_text(input_text: str) -> str:
    """Use Granite to refine/clean text before audiobook conversion."""
    inputs = granite_tokenizer(input_text, return_tensors="pt").to(granite_model.device)
    with torch.no_grad():
        outputs = granite_model.generate(
            **inputs,
            max_new_tokens=256,
            temperature=0.7,
            do_sample=True
        )
    return granite_tokenizer.decode(outputs[0], skip_special_tokens=True)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [17]:
DEFAULT_TTS_MODEL = "tts_models/en/vctk/vits"  # multi-speaker model

def init_tts(tts_model=DEFAULT_TTS_MODEL):
    print("Loading Coqui TTS:", tts_model)
    tts = TTS(model_name=tts_model, progress_bar=False)
    try:
        if torch.cuda.is_available():
            tts = tts.to("cuda")
    except Exception:
        pass
    if hasattr(tts, "speakers") and tts.speakers:
        print(f"✅ Multi-speaker model loaded. Example speakers: {tts.speakers[:5]} ...")
    else:
        print("✅ Single-speaker model loaded.")
    return tts

tts_engine = init_tts()

def synthesize_to_wav(tts, text: str, out_path: str, speaker: str = None):
    if hasattr(tts, "speakers") and tts.speakers:
        if speaker is None:
            speaker = tts.speakers[0]
            print(f"[INFO] Using default speaker: {speaker}")
        tts.tts_to_file(text=text, file_path=out_path, speaker=speaker, split_sentences=True)
    else:
        tts.tts_to_file(text=text, file_path=out_path, split_sentences=True)
    return out_path


Loading Coqui TTS: tts_models/en/vctk/vits
✅ Multi-speaker model loaded. Example speakers: ['ED\n', 'p225', 'p226', 'p227', 'p228'] ...


In [17]:
def generate_audiobook(text, out_path, speaker=None, refine=False):
    """
    Generate audiobook from text using IBM Granite (optional refinement) + Coqui TTS.
    Automatically selects first speaker for multi-speaker models.
    Saves output to out_path and returns the path.
    """
    import os

    if refine:
        try:
            text = refine_text_with_granite(text)  # make sure this function exists
        except Exception as e:
            print("⚠️ Granite refinement skipped:", e)

    if not text or text.strip() == "":
        print("❌ No text provided for audiobook generation")
        return None

    # Auto-select first speaker if multi-speaker
    try:
        available_speakers = tts_engine.speakers
        if available_speakers:
            speaker = speaker or available_speakers[0]
    except AttributeError:
        # Single-speaker model: speaker not needed
        speaker = None

    try:
        tts_engine.tts_to_file(
            text=text,
            file_path=out_path,
            speaker=speaker
        )
        if os.path.exists(out_path):
            print(f"✅ Audiobook saved: {out_path} (Speaker: {speaker})")
            return out_path
        else:
            print("❌ File not saved")
            return None
    except Exception as e:
        print("❌ Error in TTS:", e)
        return None


In [4]:
from pathlib import Path
import docx
import PyPDF2
from ebooklib import epub

def extract_text_from_file(file_path: str) -> str:
    """Extract text from txt, pdf, docx, epub."""
    file_ext = Path(file_path).suffix.lower()

    if file_ext == ".txt":
        with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
            return f.read()

    elif file_ext == ".pdf":
        text = ""
        with open(file_path, "rb") as f:
            reader = PyPDF2.PdfReader(f)
            for page in reader.pages:
                text += page.extract_text() or ""
        return text

    elif file_ext == ".docx":
        doc = docx.Document(file_path)
        return "\n".join([para.text for para in doc.paragraphs])

    elif file_ext == ".epub":
        book = epub.read_epub(file_path)
        text = []
        for item in book.get_items():
            if item.get_type() == 9:  # DOCUMENT
                try:
                    text.append(item.get_content().decode("utf-8"))
                except:
                    pass
        return "\n".join(text)

    else:
        raise ValueError("Unsupported file format. Please upload .txt, .pdf, .docx, or .epub.")


In [20]:
def handle_generation(text, refine=False):
    out_path = "audiobook.wav"
    return generate_audiobook(text, out_path, speaker=None, refine=refine)

def handle_file_upload(file, refine=False):
    if file is None:
        return None
    text = extract_text_from_file(file.name)
    out_path = "audiobook.wav"
    return generate_audiobook(text, out_path, speaker=None, refine=refine)


In [14]:
from TTS.api import TTS

# Load Coqui TTS model (multi-speaker English)
tts_engine = TTS("tts_models/en/vctk/vits")

# List available speakers
try:
    speaker_list = tts_engine.speakers
except AttributeError:
    speaker_list = ["default"]

print("✅ TTS engine loaded")
print("Available speakers:", speaker_list)


✅ TTS engine loaded
Available speakers: ['ED\n', 'p225', 'p226', 'p227', 'p228', 'p229', 'p230', 'p231', 'p232', 'p233', 'p234', 'p236', 'p237', 'p238', 'p239', 'p240', 'p241', 'p243', 'p244', 'p245', 'p246', 'p247', 'p248', 'p249', 'p250', 'p251', 'p252', 'p253', 'p254', 'p255', 'p256', 'p257', 'p258', 'p259', 'p260', 'p261', 'p262', 'p263', 'p264', 'p265', 'p266', 'p267', 'p268', 'p269', 'p270', 'p271', 'p272', 'p273', 'p274', 'p275', 'p276', 'p277', 'p278', 'p279', 'p280', 'p281', 'p282', 'p283', 'p284', 'p285', 'p286', 'p287', 'p288', 'p292', 'p293', 'p294', 'p295', 'p297', 'p298', 'p299', 'p300', 'p301', 'p302', 'p303', 'p304', 'p305', 'p306', 'p307', 'p308', 'p310', 'p311', 'p312', 'p313', 'p314', 'p316', 'p317', 'p318', 'p323', 'p326', 'p329', 'p330', 'p333', 'p334', 'p335', 'p336', 'p339', 'p340', 'p341', 'p343', 'p345', 'p347', 'p351', 'p360', 'p361', 'p362', 'p363', 'p364', 'p374', 'p376']


In [16]:
# Pick the first available speaker
speaker = speaker_list[0] if speaker_list else None

# Test TTS directly with a speaker
out_path = "test.wav"
tts_engine.tts_to_file("Hello! This is a test.", file_path=out_path, speaker=speaker)

import os
print("File exists?", os.path.exists(out_path))
print("Speaker used:", speaker)


File exists? True
Speaker used: ED



In [21]:
import gradio as gr

# ------------------------
# Handlers for Gradio
# ------------------------
def handle_generation(text, refine=False):
    out_path = "audiobook.wav"
    return generate_audiobook(text, out_path, speaker=None, refine=refine)

def handle_file_upload(file, refine=False):
    if file is None:
        return None
    text = extract_text_from_file(file.name)
    out_path = "audiobook.wav"
    return generate_audiobook(text, out_path, speaker=None, refine=refine)

# ------------------------
# Gradio UI
# ------------------------
with gr.Blocks() as demo:
    gr.Markdown("## 🎧 EchoVerse – AI Powered Audiobook Generator")

    # --- Paste Text Tab ---
    with gr.Tab("📖 Paste Text"):
        with gr.Row():
            with gr.Column():
                input_text = gr.Textbox(
                    label="Enter your text",
                    lines=10,
                    placeholder="Paste your story, article, or book chapter here..."
                )
                refine_opt1 = gr.Checkbox(
                    label="Refine text with IBM Granite", value=True
                )
                generate_btn1 = gr.Button("🎙️ Generate Audiobook")
            with gr.Column():
                audio_output1 = gr.Audio(
                    label="Generated Audiobook", type="filepath"
                )

        generate_btn1.click(
            fn=handle_generation,
            inputs=[input_text, refine_opt1],  # Only valid Gradio components
            outputs=audio_output1
        )

    # --- Upload File Tab ---
    with gr.Tab("📂 Upload File"):
        with gr.Row():
            with gr.Column():
                file_input = gr.File(
                    label="Upload a file (.txt, .pdf, .docx, .epub)",
                    file_types=[".txt", ".pdf", ".docx", ".epub"]
                )
                refine_opt2 = gr.Checkbox(
                    label="Refine text with IBM Granite", value=True
                )
                generate_btn2 = gr.Button("🎙️ Generate Audiobook from File")
            with gr.Column():
                audio_output2 = gr.Audio(
                    label="Generated Audiobook", type="filepath"
                )

        generate_btn2.click(
            fn=handle_file_upload,
            inputs=[file_input, refine_opt2],
            outputs=audio_output2
        )

demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9f97233dbd86b714a0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
